In [23]:
# Використати техніки об’єднання, злиття, транспонування і приєднання додаткових даних.
import pandas as pd
import numpy as np

diabetes_data = pd.read_csv("../data/diabetes_non_nulled.csv")

In [24]:
# ------ join ------

# Я вирішив додати якісь загальні значення відповідні до метрик з нашого датасету.
# Для цього додав табличку по BMI
bmi_ref = pd.read_csv("../data/bmi_ref.csv")
bmi_ref

,bmi_cat,bmi_min,bmi_max,severity_rank,risk_note
0,underweight,-inf,18.5,0,low risk
1,normal,18.5,25.0,1,baseline risk
2,overweight,25.0,30.0,2,elevated cardiometabolic risk
3,obesity,30.0,inf,3,high cardiometabolic risk


In [25]:
# Оскільки у нас є лише значення BMI, а для категорій потрібні діапазони, то варто підрахувати
# і тим сами створити ключ для обʼєднання
def choose_category(bmi):
    for cat in bmi_ref.index:
        if bmi < bmi_ref.loc[cat, "bmi_max"] and bmi >= bmi_ref.loc[cat, "bmi_min"]:
            return bmi_ref.loc[cat, "bmi_cat"]


diabetes_data["bmi_cat"] = diabetes_data["BMI"].apply(lambda x: choose_category(x))
diabetes_data.head(10)

# Тепер можемо обʼєднати і доповнити відповідними даними з таблички по BMI
diabetes_data_enriched = diabetes_data.merge(bmi_ref, on="bmi_cat", how="left")

# diabetes_data_enriched.groupby('risk_note')['Outcome'].sum()

In [26]:
# ------ pivot ------


# diabetes_data_enriched.pivot_table(
#     index="severity_rank",
#     columns="Outcome",
#     values="row_id",
#     aggfunc="count",
#     fill_value=0
# )

# У відсотках
(
    diabetes_data_enriched.pivot_table(
        index="severity_rank",
        columns="Outcome",
        values="row_id",
        aggfunc="count",
        fill_value=0,
    ).pipe(lambda x: round(x.div(x.sum(axis=1), axis=0) * 100, 2))
)

# Як бачимо то маємо чітку відповідність між рівнем ризику для Індексу маси тіла та наявності діабету.
# Важко сказати чи це причина чи наслідок, оскільки діабет може спиричиняти набір ваги, проте кореляція досить чітка.

Outcome,0,1
severity_rank,,
0,100.00,0.00
1,93.14,6.86
2,77.65,22.35
3,54.24,45.76


In [27]:
# ------ unstack ------

metrics = ["Glucose", "BloodPressure", "Age"]

s = diabetes_data_enriched.groupby(["severity_rank", "Outcome"])[metrics].median()
wide = s.unstack("Outcome")
print(wide)

# Glucose - видно чітку різницю між Outcome 0/1 незалежно від severity_rank.
# BloodPressure - не так сильно, але теж підвищений для групи з Outcome=1, також зростає відповідно до підняття severity_rank
# Age - в групі Outcome=1 медіана віку більша для всіх severity_rank, але не корелює з severity_rank як таким

              Glucose        BloodPressure         Age      
Outcome             0      1             0     1     0     1
severity_rank                                               
0                97.0    NaN          69.0   NaN  24.0   NaN
1               101.0  162.0          66.0  72.0  26.0  50.0
2               109.0  124.5          68.0  73.0  27.0  36.0
3               108.0  142.0          72.0  76.0  27.0  36.0


In [28]:
# concat
young = diabetes_data_enriched[diabetes_data_enriched["Age"] < 30].assign(
    age_group="young"
)
old = diabetes_data_enriched[diabetes_data_enriched["Age"] >= 30].assign(
    age_group="old"
)
combined = pd.concat([young, old], axis=0)
print(combined)
combined.groupby("age_group")["Outcome"].value_counts()

# Бачимо що вік є суттєвим ризиком діабету, так як для категорії old знаяення Outcome приблизно 50%
# Це також підтвреджує результати unstack аналізу.

     row_id  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  \
3         3            1       89           66.0             23       94   
6         6            3       78           50.0             32       88   
7         7           10      115           70.0              0        0   
20       20            3      126           88.0             41      235   
23       23            9      119           80.0             35        0   
..      ...          ...      ...            ...            ...      ...   
761     761            9      170           74.0             31        0   
762     762            9       89           62.0              0        0   
763     763           10      101           76.0             48      180   
765     765            5      121           72.0             23      112   
766     766            1      126           60.0              0        0   

      BMI  DiabetesPedigreeFunction  Age  Outcome   BMI_log  \
3    28.1               

age_group  Outcome
old        0          188
           1          184
young      0          312
           1           84
Name: count, dtype: int64

In [29]:
# melt
melted = diabetes_data_enriched.melt(
    id_vars=["row_id", "Outcome"],
    value_vars=["Glucose", "BloodPressure", "BMI"],
    var_name="metric",
    value_name="value",
)

# Порівняння медіан всіх метрик за Outcome
# Усі три показники вищі у групі з діабетом, особливо Glucose - що підтверджує його як ключевий маркер діабету (очевидно)
melted.groupby(["metric", "Outcome"])["value"].median().unstack()

Outcome,0,1
metric,,
BMI,30.1,34.3
BloodPressure,70.0,74.5
Glucose,107.0,140.0


In [30]:
# transpose
stats = diabetes_data_enriched.groupby("Outcome")[
    ["Glucose", "BloodPressure", "BMI", "Age"]
].describe()
stats.T
# За допомогою транспортування можна зручно порівняти статистики по усім показникуам, що особливо корисно коли у нас є багато метрик.

Outcome                       0           1
Glucose       count  500.000000  268.000000
              mean   110.622000  142.302239
              std     24.703920   29.488811
              min     44.000000   78.000000
              25%     93.000000  119.000000
              50%    107.000000  140.000000
              75%    125.000000  167.000000
              max    197.000000  199.000000
BloodPressure count  500.000000  268.000000
              mean    70.844000   75.272388
              std     11.928631   11.927232
              min     24.000000   30.000000
              25%     63.500000   68.000000
              50%     70.000000   74.500000
              75%     78.000000   82.000000
              max    122.000000  114.000000
BMI           count  500.000000  268.000000
              mean    30.846000   35.398507
              std      6.502089    6.590852
              min     18.200000   22.900000
              25%     25.750000   30.900000
              50%     30.100000   34.300000
              75%     35.300000   38.775000
              max     57.300000   67.100000
Age           count  500.000000  268.000000
              mean    31.190000   37.067164
              std     11.667655   10.968254
              min     21.000000   21.000000
              25%     23.000000   28.000000
              50%     27.000000   36.000000
              75%     37.000000   44.000000
              max     81.000000   70.000000